In [1]:
import json
import pandas as pd

In [2]:
business_json_path = 'yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

In [3]:
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [4]:
df_b.shape

(150346, 14)

In [5]:
# 1 = open, 0 = closed
df_b = df_b[df_b['is_open']==1]

drop_columns = ['hours','is_open','review_count']
df_b = df_b.drop(drop_columns, axis=1)

In [6]:
business_ = df_b[df_b['categories'].str.contains(
              'Restaurant|Restaurants|food',
              case=False, na=False)]

In [7]:
business_.shape

(44594, 11)

In [8]:
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

df_explode.categories.value_counts()

df_explode[df_explode.categories.str.contains('RV',
                      case=True,na=False)].categories.value_counts()

RV Dealers    103
RV Repair     103
RV Parks       82
RV Rental      54
Name: categories, dtype: int64

In [10]:
pd.__version__

'1.1.3'

In [11]:
review_json_path = 'yelp_academic_dataset_review.json'

size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [12]:
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

589820 out of 1,000,000 related reviews
593701 out of 1,000,000 related reviews
570858 out of 1,000,000 related reviews
578732 out of 1,000,000 related reviews
595538 out of 1,000,000 related reviews
589779 out of 1,000,000 related reviews
586702 out of 1,000,000 related reviews


In [13]:
csv_name = "yelp_reviews_restaurant_categories.csv"
df.to_csv(csv_name, index=False)

In [14]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,text,date
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35


In [17]:
df.city.unique()

array(['Philadelphia', 'Green Lane', 'Ashland City', 'Nashville',
       'Tampa Bay', 'Indianapolis', 'Reno', 'White House', 'Ardmore',
       'Alton', 'Edmonton', 'Bala Cynwyd', 'Tampa', 'Tucson', 'Fernley',
       'Williamstown', 'Glenolden', 'Wesley Chapel', 'Santa Barbara',
       'New Orleans', 'Camden', 'Fairview Heights', 'Wilmington',
       'Treasure Island', 'Saint Louis', 'Tarpon Springs', 'Blue Bell',
       'Brentwood', 'Woodbury', 'Brownsburg', 'Carmel', 'Greenwood',
       'Largo', 'Madison', 'Ewing', 'Aston', 'Warrington', 'Wyncote',
       'St. Louis', 'Lutz', 'Langhorne', 'King of Prussia', 'Clearwater',
       'Voorhees', 'Avon', 'Franklin', 'Meridian', 'St Albert',
       'Downingtown', 'Virginia City', 'Saint Petersburg', 'Chester',
       'Brandon', 'Exton', 'Odessa', 'Maple Shade', 'Lansdale',
       'Glenside', 'Goodlettsville', 'Narberth', 'Oldsmar', 'Bristol',
       'Glassboro', 'Spring Hill', 'Pennsville', 'Haddon Heights',
       'Goleta', 'View', 'Brookhav

In [23]:
df_ed = pd.read_csv('yelp_reviews_restaurant_categories.csv')

In [24]:
df_ed.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,text,date
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35


In [25]:
df_ed.shape

(4105130, 15)

In [29]:
df_ed1 = df_ed.loc[df_ed['city']=='Edmonton']
df_ed1 = df_ed1.drop(['state'], axis = 1)
df_ed1.head()

,business_id,name,address,city,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,text,date
1283,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",xTlqpuuqtO0FWs12gd1v6A,5,Came here to print off some documents and was ...,2014-10-15 01:49:26
1284,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",f9lq4KAus-xCsmJmjXeKVw,3,Naked Cyber & Expresso Bar is a 24 hours cyber...,2010-01-26 21:14:29
1285,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",r3QexFIhBXBT99canAgVEg,4,I visited Naked for the first time this weeken...,2013-02-05 17:01:52
1286,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",xjafuSH_qDHXEkU0LyAxvg,4,Neat cafe! Delicious sandwiches and sweets wit...,2017-05-03 16:35:44
1287,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",758g6NGLp9deCbvowz62Ww,3,"Naked is a cyber-cafe, one of the first ones t...",2008-10-22 02:56:43


In [31]:
csv_name = "yelp_edmonton_reviews_restaurant_categories.csv"
df_ed1.to_csv(csv_name, index=False)